 # Lab:  Transfer Learning with a Pre-Trained Deep Neural Network

As we discussed earlier, state-of-the-art neural networks involve millions of parameters that are prohibitively difficult to train from scratch.  In this lab, we will illustrate a powerful technique called *fine-tuning* where we start with a large pre-trained network and then re-train only the final layers to adapt to a new task.  The method is also called *transfer learning* and can produce excellent results on very small datasets with very little computational time.  

This lab is based partially on this
[excellent blog](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html).  In performing the lab, you will learn to:
* Build a custom image dataset
* Fine tune the final layers of an existing deep neural network for a new classification task.
* Load images with a `DataGenerator`.

The lab has two versions:
* *CPU version*:  In this version, you use lower resolution images so that the lab can be performed on your laptop.  The resulting accuracy is lower.  The code will also take considerable time to execute.
* *GPU version*:  This version uses higher resolution images but requires a GPU instance. See the [notes](../GCP/getting_started.md) on setting up a GPU instance on Google Cloud Platform.  The GPU training is much faster (< 1 minute).  

**MS students must complete the GPU version** of this lab.

## Create a Dataset

In this example, we will try to develop a classifier that can discriminate between two classes:  `cars` and `bicycles`.  One could imagine this type of classifier would be useful in vehicle vision systems.   The first task is to build a dataset.  

TODO:  Create training and test datasets with:
* 1000 training images of cars
* 1000 training images of bicylces
* 300 test images of cars
* 300 test images of bicylces
* The images don't need to be the same size.  But, you can reduce the resolution if you need to save disk space.

The images should be organized in the following directory structure:

    ./train
        /car
           car_0000.jpg
           car_0001.jpg
           ...
           car_0999.jpg
        /bicycle
           bicycle_0000.jpg
           bicycle_0001.jpg
           ...
           bicycle_0999.jpg
    ./test
        /car
           car_1001.jpg
           car_1001.jpg
           ...
           car_1299.jpg
        /bicycle
           bicycle_1000.jpg
           bicycle_1001.jpg
           ...
           bicycle_1299.jpg
           
The naming of the files within the directories does not matter.  The `ImageDataGenerator` class below will find the filenames.  Just make sure there are the correct number of files in each directory.
           
A nice automated way of building such a dataset if through the [FlickrAPI](demo2_flickr_images.ipynb).  Remember that if you run the FlickrAPI twice, it may collect the same images.  So, you need to run it once and split the images into training and test directories.         
        

In [1]:
import flickrapi
import urllib.request
import matplotlib.pyplot as plt
import numpy as np
import skimage.io
import skimage.transform
import requests
from io import BytesIO
from scipy.ndimage import affine_transform
import os
import warnings

%matplotlib inline

In [ ]:
api_key = u'9a1d373d19dc05fee6c917f89c5e87f5'
api_secret = u'459c530fd9f16a45'
flickr = flickrapi.FlickrAPI(api_key, api_secret)

In [ ]:
dir_name = ['./train/', './test/']
keyword_name = ['car', 'bicycle']

for dir_n in dir_name:
    for keyword_n in keyword_name:
        
        full_dir = dir_n + keyword_n + '/'

        dir_exists = os.path.isdir(full_dir)
        if not dir_exists:
            os.makedirs(full_dir)
            print("Making directory %s" % full_dir)
        else:
            print("Will store images in directory %s" % full_dir)

In [ ]:
nrow = 150
ncol = 150
for keys in keyword_name:
    photo = flickr.walk(text=keys, tag_mode='all', tags=keys,extras='url_c',\
                     sort='relevance',per_page=100)
    i = 0
    for photo in photo:
        url=photo.get('url_c')
        if not (url is None):

            # Create a file from the URL
            # This may only work in Python3
            response = requests.get(url)
            file = BytesIO(response.content)

            # Read image from file
            im = skimage.io.imread(file)

            # Resize images
            im1 = skimage.transform.resize(im,(nrow,ncol),mode='constant', anti_aliasing=True)

            # Convert to uint8, suppress the warning about the precision loss
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                im2 = skimage.img_as_ubyte(im1)
            if i < 1000:
            # Save the image
                local_name = './train/' + keys + '/' + keys + '_' + '{0:04d}.jpg'.format(i)
            else:
                local_name = './test/' + keys + '/' + keys + '_' + '{0:04d}.jpg'.format(i)
            skimage.io.imsave(local_name, im2)      
            print(local_name)
            i = i + 1       
        if (i >= 1300):        
            break        

## Loading a Pre-Trained Deep Network

We follow the [VGG16 demo](./vgg16.ipynb) to load a pre-trained deep VGG16 network.  First, run a command to verify your instance is connected to a GPU.
Now load the appropriate packages.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader, sampler, SubsetRandomSampler
from torchvision import transforms, utils
import torch.nn.functional as F

USE_GPU = True

dtype = torch.float32 # float or double

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cpu


In [3]:
import torchvision.models as models

Set the dimensions of the input image.  The sizes below would work on a GPU machine.  But, if you have a CPU image, you can use a smaller image size, like `64 x 64`.
* It seems to me that nn.AdaptiveAvgPool2d is used in pytorch implement of VGG16, which make the model adaptive to image of any size

Now we follow the [VGG16 demo](./vgg16.ipynb) and load the deep VGG16 network.  Alternatively, you can use any other pre-trained model in keras.  When using the `applications.VGG16` method you will need to:
* Set `include_top=False` to not include the top layer
* Set the `image_shape` based on the above dimensions.  Remember, `image_shape` should be height x width x 3 since the images are color.

To create now new model, we create a Sequential model.  Then, loop over the layers in `base_model.layers` and add each layer to the new model.
* I clip the classification layers in a more flexible way

Next, loop through the layers in `model`, and freeze each layer by setting `layer.trainable = False`.  This way, you will not have to *re-train* any of the existing layers.

Now, add the following layers to `model`:
* A `Flatten()` layer which reshapes the outputs to a single channel.
* A fully-connected layer with 256 output units and `relu` activation
* A final fully-connected layer.  Since this is a binary classification, there should be one output and `sigmoid` activation.

Print the model summary.  This will display the number of trainable parameters vs. the non-trainable parameters.

In [4]:
# TODO
class Flatten(nn.Module):
    '''
    A Flatten Layer implement from 
    https://gist.github.com/VoVAllen/5531c78a2d3f1ff3df772038bca37a83
    '''
    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        return x.view(x.size(0), -1)

vgg16 = models.vgg16(pretrained=True)

# Freeze the weights in VGG 16
for param in vgg16.parameters():
    param.requires_grad = False

'''
Notes: 

using feature extraction, global average pooling and add classifier to tail of it
Obviously a VGG 16 with 5 maxpool would be too much for our image size, 
we have to promise that at least 7*7 is leftover after 2 pow number of maxpool, this requires at least 224

So I remove the 7*7 adaptive average pool and use a 1*1 global average pool instead
This is a trick ResNet use to reduce image size, and it should be fine interm of performance
'''

# TODO:  Load the VGG16 network
# input_shape = ...
# base_model = applications.VGG16(weights='imagenet', ...)

model = nn.Sequential(vgg16.features,
                    nn.AdaptiveAvgPool2d((1,1)),
                    Flatten(),
                    nn.Linear(512, 512),
                    nn.BatchNorm1d(512),
                    nn.ReLU(inplace=True),
                    nn.Linear(512, 1)
                    )

print(model) # print the model summary

'''
Notes:

print the size of params that to be trained
linear1 w - 512*512
linear1 b - 512
bn mean - 512
bn std - 512
linear2 w - 1*512
linear2 b - 1
'''
for param in model.parameters():
    if param.requires_grad == True:
        print(param.size())

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

## Using Generators to Load Data

Up to now, the training data has been represented in a large matrix.  This is not possible for image data when the datasets are very large.  For these applications, the `keras` package provides a `ImageDataGenerator` class that can fetch images on the fly from a directory of images.  Using multi-threading, training can be performed on one mini-batch while the image reader can read files for the next mini-batch. The code below creates an `ImageDataGenerator` for the training data.  In addition to the reading the files, the `ImageDataGenerator` creates random deformations of the image to expand the total dataset size.  When the training data is limited, using data augmentation is very important.

In [ ]:
def AffineFun(img, r, xm, ym, order):
    sinx = np.sin(np.deg2rad(r))
    cosx = np.cos(np.deg2rad(r))

    xc = img[0].shape[0]//2
    yc = img[0].shape[1]//2

    Mc = np.array([[1, 0, xc],[0, 1, yc],[0, 0, 1]])
    R = np.array([[cosx, -sinx, 0],[sinx, cosx, 0],[0, 0, 1]])
    Mb = np.array([[1, 0,-xc],[0, 1,-yc],[0, 0, 1]])
    MM = np.array([[1, 0, xm],[0, 1, ym],[0, 0, 1]])

    Matrix = np.linalg.multi_dot([Mc, R, Mb, MM])

    C = img.shape[0]
    for chan in np.arange(C):
        img[chan] = affine_transform(img[chan], Matrix, output_shape=img[chan].shape, order=order)

    return img

def filpFun(img, x):

    if x==True:
        img = np.flip(img, axis=2)

    return img

class RandomFilp(object):
    def __init__(self, p):
        self.p = p

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        return {'image': filpFun(image, self.p), 'label': label}

class RandomAffine(object):
    def __init__(self, fluR, fluM):

        self.fluR = fluR
        self.fluM = fluM

    def __call__(self, sample):

        r = np.random.uniform(-self.fluR, self.fluR, size=1)
        xm, ym = np.random.uniform(-self.fluM, self.fluM, size=2)

        image, label = sample['image'], sample['label']
        return {'image': AffineFun(image, r, xm, ym, 3), \
                'label': label}

def toTensor(sample):
    image, label = sample['image'], sample['label']
    return {'image': torch.from_numpy(image.copy()),
            'label': torch.from_numpy(label.copy())}

class CBDataset(Dataset):
    """car, bicycle dataset."""

    def __init__(self, transform=None):
        self.transform = transform
        pass
    
    def __len__(self):
        return 2600

    def __getitem__(self, idx):
        # idx \in [0, 2600)
        # car, bicycle in [0, 1299)
        if idx%2 == 0:
            image = skimage.io.imread('./train/car/' + 'car' + '_' + '{0:04d}.jpg'.format(idx//2))
            label = np.array([0])
        else:
            image = skimage.io.imread('./train/bicycle/' + 'bicycle' + '_' + '{0:04d}.jpg'.format(idx//2))
            label = np.array([1])
        
        if image.ndim == 2:
            # black and white
            image = image.reshape(*image.shape, 1)
            image = np.concatenate((image, image, image), axis=2)
        
        image = image.transpose((2,0,1))
        sample = {'image': image, 'label': label}
        
        if self.transform:
            sample = self.transform(sample)
        
        # for showing purpose only, comment after debug
        # plt.imshow(sample['image'].transpose(1, 2, 0))
        # plt.show()
        
        sample = toTensor(sample)

        return sample
    
transformed_dataset = CBDataset(
                        transform=transforms.Compose([
                            RandomFilp(True),
                            RandomAffine(10, 20)
                        ])
                    )

for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]
    print(i, sample['image'].size(), sample['label'].size())

    if i == 2600:
        break

0 torch.Size([3, 150, 150]) torch.Size([1])
1 torch.Size([3, 150, 150]) torch.Size([1])
2 torch.Size([3, 150, 150]) torch.Size([1])
3 torch.Size([3, 150, 150]) torch.Size([1])
4 torch.Size([3, 150, 150]) torch.Size([1])
5 torch.Size([3, 150, 150]) torch.Size([1])
6 torch.Size([3, 150, 150]) torch.Size([1])
7 torch.Size([3, 150, 150]) torch.Size([1])
8 torch.Size([3, 150, 150]) torch.Size([1])
9 torch.Size([3, 150, 150]) torch.Size([1])
10 torch.Size([3, 150, 150]) torch.Size([1])
11 torch.Size([3, 150, 150]) torch.Size([1])
12 torch.Size([3, 150, 150]) torch.Size([1])
13 torch.Size([3, 150, 150]) torch.Size([1])
14 torch.Size([3, 150, 150]) torch.Size([1])
15 torch.Size([3, 150, 150]) torch.Size([1])
16 torch.Size([3, 150, 150]) torch.Size([1])
17 torch.Size([3, 150, 150]) torch.Size([1])
18 torch.Size([3, 150, 150]) torch.Size([1])
19 torch.Size([3, 150, 150]) torch.Size([1])
20 torch.Size([3, 150, 150]) torch.Size([1])
21 torch.Size([3, 150, 150]) torch.Size([1])
22 torch.Size([3, 15

180 torch.Size([3, 150, 150]) torch.Size([1])
181 torch.Size([3, 150, 150]) torch.Size([1])
182 torch.Size([3, 150, 150]) torch.Size([1])
183 torch.Size([3, 150, 150]) torch.Size([1])
184 torch.Size([3, 150, 150]) torch.Size([1])
185 torch.Size([3, 150, 150]) torch.Size([1])
186 torch.Size([3, 150, 150]) torch.Size([1])
187 torch.Size([3, 150, 150]) torch.Size([1])
188 torch.Size([3, 150, 150]) torch.Size([1])
189 torch.Size([3, 150, 150]) torch.Size([1])
190 torch.Size([3, 150, 150]) torch.Size([1])
191 torch.Size([3, 150, 150]) torch.Size([1])
192 torch.Size([3, 150, 150]) torch.Size([1])
193 torch.Size([3, 150, 150]) torch.Size([1])
194 torch.Size([3, 150, 150]) torch.Size([1])
195 torch.Size([3, 150, 150]) torch.Size([1])
196 torch.Size([3, 150, 150]) torch.Size([1])
197 torch.Size([3, 150, 150]) torch.Size([1])
198 torch.Size([3, 150, 150]) torch.Size([1])
199 torch.Size([3, 150, 150]) torch.Size([1])
200 torch.Size([3, 150, 150]) torch.Size([1])
201 torch.Size([3, 150, 150]) torc

358 torch.Size([3, 150, 150]) torch.Size([1])
359 torch.Size([3, 150, 150]) torch.Size([1])
360 torch.Size([3, 150, 150]) torch.Size([1])
361 torch.Size([3, 150, 150]) torch.Size([1])
362 torch.Size([3, 150, 150]) torch.Size([1])
363 torch.Size([3, 150, 150]) torch.Size([1])
364 torch.Size([3, 150, 150]) torch.Size([1])
365 torch.Size([3, 150, 150]) torch.Size([1])
366 torch.Size([3, 150, 150]) torch.Size([1])
367 torch.Size([3, 150, 150]) torch.Size([1])
368 torch.Size([3, 150, 150]) torch.Size([1])
369 torch.Size([3, 150, 150]) torch.Size([1])
370 torch.Size([3, 150, 150]) torch.Size([1])
371 torch.Size([3, 150, 150]) torch.Size([1])
372 torch.Size([3, 150, 150]) torch.Size([1])
373 torch.Size([3, 150, 150]) torch.Size([1])
374 torch.Size([3, 150, 150]) torch.Size([1])
375 torch.Size([3, 150, 150]) torch.Size([1])
376 torch.Size([3, 150, 150]) torch.Size([1])
377 torch.Size([3, 150, 150]) torch.Size([1])
378 torch.Size([3, 150, 150]) torch.Size([1])
379 torch.Size([3, 150, 150]) torc

536 torch.Size([3, 150, 150]) torch.Size([1])
537 torch.Size([3, 150, 150]) torch.Size([1])
538 torch.Size([3, 150, 150]) torch.Size([1])
539 torch.Size([3, 150, 150]) torch.Size([1])
540 torch.Size([3, 150, 150]) torch.Size([1])
541 torch.Size([3, 150, 150]) torch.Size([1])
542 torch.Size([3, 150, 150]) torch.Size([1])
543 torch.Size([3, 150, 150]) torch.Size([1])
544 torch.Size([3, 150, 150]) torch.Size([1])
545 torch.Size([3, 150, 150]) torch.Size([1])
546 torch.Size([3, 150, 150]) torch.Size([1])
547 torch.Size([3, 150, 150]) torch.Size([1])
548 torch.Size([3, 150, 150]) torch.Size([1])
549 torch.Size([3, 150, 150]) torch.Size([1])
550 torch.Size([3, 150, 150]) torch.Size([1])
551 torch.Size([3, 150, 150]) torch.Size([1])
552 torch.Size([3, 150, 150]) torch.Size([1])
553 torch.Size([3, 150, 150]) torch.Size([1])
554 torch.Size([3, 150, 150]) torch.Size([1])
555 torch.Size([3, 150, 150]) torch.Size([1])
556 torch.Size([3, 150, 150]) torch.Size([1])
557 torch.Size([3, 150, 150]) torc

714 torch.Size([3, 150, 150]) torch.Size([1])
715 torch.Size([3, 150, 150]) torch.Size([1])
716 torch.Size([3, 150, 150]) torch.Size([1])
717 torch.Size([3, 150, 150]) torch.Size([1])
718 torch.Size([3, 150, 150]) torch.Size([1])
719 torch.Size([3, 150, 150]) torch.Size([1])
720 torch.Size([3, 150, 150]) torch.Size([1])
721 torch.Size([3, 150, 150]) torch.Size([1])
722 torch.Size([3, 150, 150]) torch.Size([1])
723 torch.Size([3, 150, 150]) torch.Size([1])
724 torch.Size([3, 150, 150]) torch.Size([1])
725 torch.Size([3, 150, 150]) torch.Size([1])
726 torch.Size([3, 150, 150]) torch.Size([1])
727 torch.Size([3, 150, 150]) torch.Size([1])
728 torch.Size([3, 150, 150]) torch.Size([1])
729 torch.Size([3, 150, 150]) torch.Size([1])
730 torch.Size([3, 150, 150]) torch.Size([1])
731 torch.Size([3, 150, 150]) torch.Size([1])
732 torch.Size([3, 150, 150]) torch.Size([1])
733 torch.Size([3, 150, 150]) torch.Size([1])
734 torch.Size([3, 150, 150]) torch.Size([1])
735 torch.Size([3, 150, 150]) torc

...

1416 torch.Size([3, 150, 150]) torch.Size([1])
1417 torch.Size([3, 150, 150]) torch.Size([1])
1418 torch.Size([3, 150, 150]) torch.Size([1])
1419 torch.Size([3, 150, 150]) torch.Size([1])
1420 torch.Size([3, 150, 150]) torch.Size([1])
1421 torch.Size([3, 150, 150]) torch.Size([1])
1422 torch.Size([3, 150, 150]) torch.Size([1])
1423 torch.Size([3, 150, 150]) torch.Size([1])
1424 torch.Size([3, 150, 150]) torch.Size([1])
1425 torch.Size([3, 150, 150]) torch.Size([1])
1426 torch.Size([3, 150, 150]) torch.Size([1])
1427 torch.Size([3, 150, 150]) torch.Size([1])
1428 torch.Size([3, 150, 150]) torch.Size([1])
1429 torch.Size([3, 150, 150]) torch.Size([1])
1430 torch.Size([3, 150, 150]) torch.Size([1])
1431 torch.Size([3, 150, 150]) torch.Size([1])
1432 torch.Size([3, 150, 150]) torch.Size([1])
1433 torch.Size([3, 150, 150]) torch.Size([1])
1434 torch.Size([3, 150, 150]) torch.Size([1])
1435 torch.Size([3, 150, 150]) torch.Size([1])
1436 torch.Size([3, 150, 150]) torch.Size([1])
1437 torch.Si

1590 torch.Size([3, 150, 150]) torch.Size([1])
1591 torch.Size([3, 150, 150]) torch.Size([1])
1592 torch.Size([3, 150, 150]) torch.Size([1])
1593 torch.Size([3, 150, 150]) torch.Size([1])
1594 torch.Size([3, 150, 150]) torch.Size([1])
1595 torch.Size([3, 150, 150]) torch.Size([1])
1596 torch.Size([3, 150, 150]) torch.Size([1])
1597 torch.Size([3, 150, 150]) torch.Size([1])
1598 torch.Size([3, 150, 150]) torch.Size([1])
1599 torch.Size([3, 150, 150]) torch.Size([1])
1600 torch.Size([3, 150, 150]) torch.Size([1])
1601 torch.Size([3, 150, 150]) torch.Size([1])
1602 torch.Size([3, 150, 150]) torch.Size([1])
1603 torch.Size([3, 150, 150]) torch.Size([1])
1604 torch.Size([3, 150, 150]) torch.Size([1])
1605 torch.Size([3, 150, 150]) torch.Size([1])
1606 torch.Size([3, 150, 150]) torch.Size([1])
1607 torch.Size([3, 150, 150]) torch.Size([1])
1608 torch.Size([3, 150, 150]) torch.Size([1])
1609 torch.Size([3, 150, 150]) torch.Size([1])
1610 torch.Size([3, 150, 150]) torch.Size([1])
1611 torch.Si

1764 torch.Size([3, 150, 150]) torch.Size([1])
1765 torch.Size([3, 150, 150]) torch.Size([1])
1766 torch.Size([3, 150, 150]) torch.Size([1])
1767 torch.Size([3, 150, 150]) torch.Size([1])
1768 torch.Size([3, 150, 150]) torch.Size([1])
1769 torch.Size([3, 150, 150]) torch.Size([1])
1770 torch.Size([3, 150, 150]) torch.Size([1])
1771 torch.Size([3, 150, 150]) torch.Size([1])
1772 torch.Size([3, 150, 150]) torch.Size([1])
1773 torch.Size([3, 150, 150]) torch.Size([1])
1774 torch.Size([3, 150, 150]) torch.Size([1])
1775 torch.Size([3, 150, 150]) torch.Size([1])
1776 torch.Size([3, 150, 150]) torch.Size([1])
1777 torch.Size([3, 150, 150]) torch.Size([1])
1778 torch.Size([3, 150, 150]) torch.Size([1])
1779 torch.Size([3, 150, 150]) torch.Size([1])
1780 torch.Size([3, 150, 150]) torch.Size([1])
1781 torch.Size([3, 150, 150]) torch.Size([1])
1782 torch.Size([3, 150, 150]) torch.Size([1])
1783 torch.Size([3, 150, 150]) torch.Size([1])
1784 torch.Size([3, 150, 150]) torch.Size([1])
1785 torch.Si

1938 torch.Size([3, 150, 150]) torch.Size([1])
1939 torch.Size([3, 150, 150]) torch.Size([1])
1940 torch.Size([3, 150, 150]) torch.Size([1])
1941 torch.Size([3, 150, 150]) torch.Size([1])
1942 torch.Size([3, 150, 150]) torch.Size([1])
1943 torch.Size([3, 150, 150]) torch.Size([1])
1944 torch.Size([3, 150, 150]) torch.Size([1])
1945 torch.Size([3, 150, 150]) torch.Size([1])
1946 torch.Size([3, 150, 150]) torch.Size([1])
1947 torch.Size([3, 150, 150]) torch.Size([1])
1948 torch.Size([3, 150, 150]) torch.Size([1])
1949 torch.Size([3, 150, 150]) torch.Size([1])
1950 torch.Size([3, 150, 150]) torch.Size([1])
1951 torch.Size([3, 150, 150]) torch.Size([1])
1952 torch.Size([3, 150, 150]) torch.Size([1])
1953 torch.Size([3, 150, 150]) torch.Size([1])
1954 torch.Size([3, 150, 150]) torch.Size([1])
1955 torch.Size([3, 150, 150]) torch.Size([1])
1956 torch.Size([3, 150, 150]) torch.Size([1])
1957 torch.Size([3, 150, 150]) torch.Size([1])
1958 torch.Size([3, 150, 150]) torch.Size([1])
1959 torch.Si

2112 torch.Size([3, 150, 150]) torch.Size([1])
2113 torch.Size([3, 150, 150]) torch.Size([1])
2114 torch.Size([3, 150, 150]) torch.Size([1])
2115 torch.Size([3, 150, 150]) torch.Size([1])
2116 torch.Size([3, 150, 150]) torch.Size([1])
2117 torch.Size([3, 150, 150]) torch.Size([1])
2118 torch.Size([3, 150, 150]) torch.Size([1])
2119 torch.Size([3, 150, 150]) torch.Size([1])
2120 torch.Size([3, 150, 150]) torch.Size([1])
2121 torch.Size([3, 150, 150]) torch.Size([1])
2122 torch.Size([3, 150, 150]) torch.Size([1])
2123 torch.Size([3, 150, 150]) torch.Size([1])
2124 torch.Size([3, 150, 150]) torch.Size([1])
2125 torch.Size([3, 150, 150]) torch.Size([1])
2126 torch.Size([3, 150, 150]) torch.Size([1])
2127 torch.Size([3, 150, 150]) torch.Size([1])
2128 torch.Size([3, 150, 150]) torch.Size([1])
2129 torch.Size([3, 150, 150]) torch.Size([1])
2130 torch.Size([3, 150, 150]) torch.Size([1])
2131 torch.Size([3, 150, 150]) torch.Size([1])
2132 torch.Size([3, 150, 150]) torch.Size([1])
2133 torch.Si

In [ ]:
## show a small portation of image
for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]
    print(i, sample['image'].size(), sample['label'].size())

    if i == 3:
        break

In [ ]:
BATCH_SIZE = 2
NUM_WORKERS = 2

train_loader = DataLoader(transformed_dataset, batch_size=BATCH_SIZE, \
                    sampler=sampler.SubsetRandomSampler(range(2000)),\
                    num_workers=NUM_WORKERS)
validation_loader = DataLoader(transformed_dataset, batch_size=BATCH_SIZE,
                    sampler=sampler.SubsetRandomSampler(range(2000,2600)),\
                    num_workers=NUM_WORKERS)

Now, create a similar `test_generator` for the test data.

In [ ]:
for i_batch, sample_batched in enumerate(train_loader):
    print(i_batch, sample_batched['image'].size(), \
          sample_batched['label'].size())
    # observe 4th batch and stop.
    if i_batch == 3:
        break

In [ ]:
# TODO
# test_generator = ...

The following function displays images that will be useful below.

In [ ]:
# Display the image


To see how the `train_generator` works, use the `train_generator.next()` method to get a minibatch of data `X,y`.  Display the first 8 images in this mini-batch and label the image with the class label.  You should see that bicycles have `y=0` and cars have `y=1`.

In [ ]:
# TODO

## Train the Model

Compile the model.  Select the correct `loss` function, `optimizer` and `metrics`.  Remember that we are performing binary classification.

In [ ]:
# TODO.
# model.compile(...)

When using an `ImageDataGenerator`, we have to set two parameters manually:
* `steps_per_epoch =  training data size // batch_size`
* `validation_steps =  test data size // batch_size`

We can obtain the training and test data size from `train_generator.n` and `test_generator.n`, respectively.

In [ ]:
# TODO

Now, we run the fit.  If you are using a CPU on a regular laptop, each epoch will take about 3-4 minutes, so you should be able to finish 5 epochs or so within 20 minutes.  On a reasonable GPU, even with the larger images, it will take about 10 seconds per epoch.
* If you use `(nrow,ncol) = (64,64)` images, you should get around 90% accuracy after 5 epochs.
* If you use `(nrow,ncol) = (150,150)` images, you should get around 96% accuracy after 5 epochs.  But, this will need a GPU.

You will get full credit for either version.  With more epochs, you may get slightly higher, but you will have to play with the damping.

Remember to record the history of the fit, so that you can plot the training and validation accuracy curve.

In [ ]:
nepochs = 5  # Number of epochs

# Call the fit_generator function
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=nepochs,
    validation_data=test_generator,
    validation_steps=validation_steps)

In [ ]:
# Plot the training accuracy and validation accuracy curves on the same figure.

# TO DO

## Plotting the Error Images

Now try to plot some images that were in error:

*  Generate a mini-batch `Xts,yts` from the `test_generator.next()` method
*  Get the class probabilities using the `model.predict( )` method and compute predicted labels `yhat`.
*  Get the images where `yts[i] ~= yhat[i]`.
*  If you did not get any prediction error in one minibatch, run it multiple times.
*  After you a get a few error images (say 4-8), plot the error images with the true labels and class probabilities predicted by the classifie

In [ ]:
# TO DO